# Pandora Auxiliary Targets Data Processing

This notebook processes the Pandora auxiliary targets list by:
1. Loading the base CSV file
2. Importing AP stars from CDS catalog (with duplicate checking)
3. Filling missing stellar data from SIMBAD
4. Filling missing planet parameters from NASA Exoplanet Archive
5. Filling proper motions from Gaia DR3
6. Saving the final updated CSV

The notebook is idempotent - you can run it multiple times on the same (or updated) CSV file without duplicating data or re-querying already populated fields.

In [ ]:
# Stupid hack for VScode, because Microsoft is dumb.
import os
os.chdir('/home/rowe/python/Pandora')
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/rowe/python/Pandora


In [ ]:
import pandas as pd
import numpy as np

## Load CSV File

Load the Pandora Snapshot Targets CSV file into a pandas DataFrame.

In [ ]:


# Read the CSV file
# df = pd.read_csv('Pandora Snapshot Targets - Targets.csv')
df = pd.read_csv('Pandora Snapshot Targets - V12.csv')
df.head()

,Target Name,Other name,DR3,Planet ID,RA (deg),DEC (deg),dRA (mas/yr),dDEC (mas/yr),Epoch,J_mag,...,T_obs (hours),T_tot (hours),VISDA Cadence,NIRDA Cadence,Priority,Program ID,Contact Person,Observational Constraints (if any),Comments,AUX Team Validated?
0,HD 209458,NaN,NaN,b,330.052992,18.884319,29.766,-17.9760,J2000,6.591,...,NaN,NaN,10s / 9 ROI (default),default,1.0,Famous Planets,J. Rowe,Needs consecutive transits,Somewhat famous planet,Yes
1,KIC 8462852,Tabby's star,NaN,NaN,300.104292,4.456886,-10.375,-10.2730,J2000,10.763,...,NaN,NaN,NaN,NaN,2.0,Exotic-Stars,J. Rowe,NaN,Eruptive Variable,NaN
2,HD 189733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,Famous Planets,J. Rowe,NaN,NaN,NaN
3,GJ 411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TOI-2095,NaN,NaN,0.03,285.636524,75.418514,203.440,-21.3176,J2000,9.797,...,NaN,NaN,NaN,NaN,2.0,TOI-2095 TTVs,E. Gilbert,NaN,"Candidate planet, discovery paper in prep per ...",NaN


## Import AP Stars Catalog

Import AP stars from CDS catalog and add them to the dataframe if not already present.

In [ ]:
# Import AP stars catalogue from CDS
import urllib.request
from io import StringIO

# Check if AP stars are already in the dataframe
ap_already_loaded = (df['Program ID'] == 'ApStarSurvey').sum() > 0

if ap_already_loaded:
    print(f"AP stars already present in dataframe ({(df['Program ID'] == 'ApStarSurvey').sum()} stars)")
    print("Skipping AP star import to avoid duplicates.")
else:
    # Download the catalogue data
    url = 'https://cdsarc.cds.unistra.fr/ftp/J/ApJ/943/147/tablea1.dat'
    print("Downloading AP stars catalogue...")
    response = urllib.request.urlopen(url)
    data_text = response.read().decode('utf-8')
    print(f"Downloaded {len(data_text)} bytes")
    
    # Parse the fixed-width format according to ReadMe
    # Bytes Format Units   Label     Explanations
    # 57- 62  F6.2  deg     RAdeg     Right Ascension (J2000)
    # 64- 68  F5.2  deg     DEdeg     Declination (J2000)
    # 77- 81  F5.2  mag     GMag      Gaia DR3 G band magnitude
    # 272-290 I19   ---     GaiaDR3   Gaia DR3 identifier
    
    ap_stars = []
    for line in data_text.strip().split('\n'):
        if len(line) < 290:  # Skip short lines
            continue
        
        try:
            # Extract using byte positions (Python uses 0-based indexing)
            ra = float(line[56:62].strip())  # bytes 57-62
            dec = float(line[63:68].strip())  # bytes 64-68
            gmag = float(line[76:81].strip())  # bytes 77-81 (using as J_mag proxy)
            gaia_dr3 = line[271:290].strip()  # bytes 272-290
            
            # Use first part of line as name (SpecID, bytes 1-39)
            name = line[0:39].strip()
            
            ap_stars.append({
                'Target Name': name,
                'Other name': gaia_dr3,
                'Planet ID': '',
                'RA (deg)': ra,
                'DEC (deg)': dec,
                'dRA (mas/yr)': np.nan,  # Will be filled by Gaia query
                'dDEC (mas/yr)': np.nan,  # Will be filled by Gaia query
                'Epoch': 'J2000',
                'J_mag': gmag,  # Using G mag as proxy
                'Number of Visits': -1,
                'IsAPlanet?': '',
                'Period (days)': np.nan,
                'P_sig': np.nan,
                'T_mid (BJD)': np.nan,
                'T0_sig': np.nan,
                'T_dur (hours)': np.nan,
                'T_obs (hours)': np.nan,
                'T_tot (hours)': np.nan,
                'VISDA Cadence': '',
                'NIRDA Cadence': '',
                'Priority': 3,
                'Program ID': 'ApStarSurvey',
                'Contact Person': 'J. Rowe',
                'Observational Constraints (if any)': '',
                'Comments': 'AP star',
                'AUX Team Validated?': ''
            })
        except (ValueError, IndexError) as e:
            print(f"Skipping line (parse error): {e}")
            continue
    
    ap_df = pd.DataFrame(ap_stars)
    
    # Append AP stars to the main dataframe
    df = pd.concat([df, ap_df], ignore_index=True)
    
    print(f"\nLoaded and added {len(ap_df)} AP stars")
    print(f"Total rows in dataframe: {len(df)}")
    print("\nFirst few AP star entries:")
    print(ap_df[['Target Name', 'Other name', 'RA (deg)', 'DEC  (deg)', 'J_mag']].head())

AP stars already present in dataframe (2710 stars)
Skipping AP star import to avoid duplicates.


## Fill Missing Stellar Data from SIMBAD

Query SIMBAD for missing coordinates, proper motions, and J magnitudes.

In [ ]:
# First, let's check which rows have missing RA, DEC, or other data
missing_coords = df[df['RA (deg)'].isna() | df['DEC (deg)'].isna() | df['dRA (mas/yr)'].isna() | df['dDEC (mas/yr)'].isna()]
print(f"Rows with missing coordinates: {len(missing_coords)}")
print("\nTarget names with missing data:")
print(missing_coords['Target Name'].tolist())

Rows with missing coordinates: 23

Target names with missing data:
['HD 189733', 'GJ 411', 'J04144730+2646264', 'J04555605+3036209', 'J05075496+2500156', 'J04385859+2336351', 'J04190110+2819420', 'J04161210+2756385', 'J04322210+1827426', 'J04334465+2615005', 'J04393364+2359212', 'J04394488+2601527', 'J04202555+2700355', 'J04284263+2714039', 'J04213459+2701388', 'J04181710+2828419', 'J04230607+2801194', 'J04262939+2624137', 'J04292165+2701259', 'J04390163+2336029', 'J04400067+2358211', 'J11145133-2618235', 'spec-56627-GAC056N46V1_sp05-214']


In [ ]:
# Install and import astroquery
from astroquery.simbad import Simbad
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np

# Configure Simbad to return the fields we need
custom_simbad = Simbad()
custom_simbad.add_votable_fields('pmra', 'pmdec', 'J')

In [ ]:
def query_simbad_for_target(target_name):
    """
    Query SIMBAD for a target and return RA, DEC, proper motions, and J magnitude.
    Returns a dict with the values or None if not found.
    """
    try:
        result = custom_simbad.query_object(target_name)
        if result is None or len(result) == 0:
            print(f"  No results found for {target_name}")
            return None
        
        # Get coordinates
        coords = SkyCoord.from_name(target_name)
        
        # Extract proper motions - check for different possible column names
        pmra = np.nan
        pmdec = np.nan
        jmag = np.nan
        
        # Try different column name variations for proper motions
        if 'PMRA' in result.colnames:
            pmra = result['PMRA'][0] if not np.ma.is_masked(result['PMRA'][0]) else np.nan
        elif 'pmra' in result.colnames:
            pmra = result['pmra'][0] if not np.ma.is_masked(result['pmra'][0]) else np.nan
            
        if 'PMDEC' in result.colnames:
            pmdec = result['PMDEC'][0] if not np.ma.is_masked(result['PMDEC'][0]) else np.nan
        elif 'pmdec' in result.colnames:
            pmdec = result['pmdec'][0] if not np.ma.is_masked(result['pmdec'][0]) else np.nan
        
        # Try different column name variations for J magnitude
        if 'FLUX_J' in result.colnames:
            jmag = result['FLUX_J'][0] if not np.ma.is_masked(result['FLUX_J'][0]) else np.nan
        elif 'J' in result.colnames:
            jmag = result['J'][0] if not np.ma.is_masked(result['J'][0]) else np.nan
        
        # Extract data from result
        data = {
            'RA (deg)': coords.ra.deg,
            'DEC (deg)': coords.dec.deg,
            'dRA (mas/yr)': pmra,
            'dDEC (mas/yr)': pmdec,
            'Epoch': 'J2000',  # SIMBAD typically uses J2000
            'J_mag': jmag
        }
        
        print(f"  Found data for {target_name}")
        return data
        
    except Exception as e:
        print(f"  Error querying {target_name}: {e}")
        return None

In [ ]:
# Fill in missing data for each row
print("Querying SIMBAD for missing data...\n")

updated_count = 0
skipped_count = 0
skipped_ap_stars = 0

for idx, row in df.iterrows():
    # Skip AP stars - they'll get data from Gaia instead
    if row['Program ID'] == 'ApStarSurvey':
        skipped_ap_stars += 1
        continue
    
    # Check if RA, DEC, proper motions, or J_mag is missing
    if pd.isna(row['RA (deg)']) or pd.isna(row['DEC (deg)'])  or pd.isna(row['dRA (mas/yr)']) or pd.isna(row['dDEC (mas/yr)']) or pd.isna(row['J_mag']):
        target_name = row['Target Name']
        
        # Skip empty target names
        if pd.isna(target_name) or target_name.strip() == '':
            continue
            
        print(f"Processing: {target_name}")
        
        # Try with the target name first
        data = query_simbad_for_target(target_name)
        
        # If that fails and there's an "Other name", try that
        if data is None and pd.notna(row['Other name']) and row['Other name'].strip() != '':
            other_name = row['Other name']
            print(f"  Trying alternate name: {other_name}")
            data = query_simbad_for_target(other_name)
        
        # Update the dataframe if we found data
        if data is not None:
            for key, value in data.items():
                if pd.isna(row[key]) and not pd.isna(value):
                    df.at[idx, key] = value
            updated_count += 1
    else:
        skipped_count += 1
                    
print(f"\nDone! Updated {updated_count} rows with SIMBAD data.")
print(f"Skipped {skipped_count} rows that already had coordinates.")
print(f"Skipped {skipped_ap_stars} AP stars (will be updated from Gaia).")

Querying SIMBAD for missing data...

Processing: HD 189733
  Found data for HD 189733
Processing: GJ 411
  Found data for GJ 411
Processing: WOW (Positive Horn)
  Found data for HD 189733
Processing: GJ 411
  Found data for GJ 411
Processing: WOW (Positive Horn)


  No results found for WOW (Positive Horn)
Processing: WOW (Positive Horn)
  No results found for WOW (Positive Horn)
Processing: Neptune
  No results found for Neptune
  Trying alternate name: Solar System


  No results found for Solar System
Processing: Uranus
  No results found for Uranus
  Trying alternate name: Solar System
  No results found for Solar System
Processing: J04144730+2646264


  No results found for J04144730+2646264
Processing: J04555605+3036209
  No results found for J04555605+3036209
Processing: J05075496+2500156


  No results found for J05075496+2500156
Processing: J04385859+2336351
  No results found for J04385859+2336351
Processing: J04190110+2819420


...

  No results found for J04213459+2701388
Processing: J04181710+2828419
  No results found for J04181710+2828419
Processing: J04230607+2801194


  No results found for J04230607+2801194
Processing: J04262939+2624137
  No results found for J04262939+2624137
Processing: J04292165+2701259


  No results found for J04292165+2701259
Processing: J04390163+2336029
  No results found for J04390163+2336029
Processing: J04400067+2358211


  No results found for J04400067+2358211
Processing: J11145133-2618235
  No results found for J11145133-2618235
  Trying alternate name: J11145133-2618235 A


  No results found for J11145133-2618235 A
Processing: TIC 1674663308
  Found data for TIC 1674663308
Processing: TIC 1101124559
  Found data for TIC 1101124559
Processing: TIC 1277142190
  Found data for TIC 1277142190
Processing: TIC 1674663309
  Found data for TIC 1101124559
Processing: TIC 1277142190
  Found data for TIC 1277142190
Processing: TIC 1674663309
  Found data for TIC 1674663309

Done! Updated 6 rows with SIMBAD data.
Skipped 318 rows that already had coordinates.
Skipped 2710 AP stars (will be updated from Gaia).
  Found data for TIC 1674663309

Done! Updated 6 rows with SIMBAD data.
Skipped 318 rows that already had coordinates.
Skipped 2710 AP stars (will be updated from Gaia).


In [ ]:
# Check the results - show rows that were updated
updated_rows = df[df['Target Name'].isin(missing_coords['Target Name'])]
print("Updated data:")
print(updated_rows[['Target Name', 'Other name', 'RA (deg)', 'DEC (deg)', 'dRA (mas/yr)', 'dDEC (mas/yr)', 'J_mag']].head(20))

Updated data:
          Target Name Other name    RA (deg)  DEC (deg)  dRA (mas/yr)  \
2           HD 189733        NaN  300.182137  22.710854        -3.208   
3              GJ 411        NaN  165.834145  35.969882      -580.057   
78  J04144730+2646264        NaN   63.697121  26.773956           NaN   
79  J04555605+3036209        NaN   73.983562  30.605822           NaN   
80  J05075496+2500156        NaN   76.979025  25.004336           NaN   
81  J04385859+2336351        NaN   69.744163  23.609767           NaN   
82  J04190110+2819420        NaN   64.754608  28.328347           NaN   
83  J04161210+2756385        NaN   64.050433  27.944050           NaN   
84  J04322210+1827426        NaN   68.092121  18.461844           NaN   
85  J04334465+2615005        NaN   68.436050  26.250147           NaN   
86  J04393364+2359212        NaN   69.890187  23.989231           NaN   
87  J04394488+2601527        NaN   69.936995  26.031331           NaN   
88  J04202555+2700355        NaN   65

In [ ]:
# Query NASA Exoplanet Archive for planet candidates with missing period/T0 data
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive

# Check which rows have planet IDs but missing period or T0 data
planet_rows = df[df['Planet ID'].notna() & (df['Planet ID'] != '')]
missing_planet_data = planet_rows[(planet_rows['Period (days)'].isna()) | (planet_rows['T_mid (BJD)'].isna())]

print(f"Rows with Planet IDs: {len(planet_rows)}")
print(f"Rows with missing period or T0: {len(missing_planet_data)}")
print("\nTargets with missing planet data:")
for idx, row in missing_planet_data.iterrows():
    print(f"  {row['Target Name']} {row['Planet ID']}")

Rows with Planet IDs: 58
Rows with missing period or T0: 2

Targets with missing planet data:
  TOI-2095 0.03
  55 Cnc e


## Fill Missing Planet Data from NASA Exoplanet Archive

Query NASA Exoplanet Archive for missing planet orbital parameters (period, T0).

In [ ]:
# Debug: Let's see the actual data for TOI-700 b
try:
    ps_result = NasaExoplanetArchive.query_object('TOI-700 b', table='ps')
    if ps_result is not None and len(ps_result) > 0:
        print(f"Found {len(ps_result)} results for TOI-700 b")
        print("\nAll available columns:")
        print(ps_result.colnames)
        print("\nFirst row data (relevant fields):")
        for col in ['pl_name', 'pl_letter', 'pl_orbper', 'pl_orbpererr1', 'pl_tranmid', 'pl_tranmiderr1']:
            if col in ps_result.colnames:
                val = ps_result[col][0]
                print(f"  {col}: {val} (type: {type(val).__name__})")
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Found 4 results for TOI-700 b

All available columns:
['pl_name', 'pl_letter', 'hostname', 'hd_name', 'hip_name', 'tic_id', 'gaia_dr2_id', 'gaia_dr3_id', 'default_flag', 'pl_refname', 'sy_refname', 'disc_pubdate', 'disc_year', 'discoverymethod', 'disc_locale', 'disc_facility', 'disc_instrument', 'disc_telescope', 'disc_refname', 'ra', 'rastr', 'dec', 'decstr', 'glon', 'glat', 'elon', 'elat', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_orbperlim', 'pl_orbperstr', 'pl_orblpererr1', 'pl_orblper', 'pl_orblpererr2', 'pl_orblperlim', 'pl_orblperstr', 'pl_orbsmax', 'pl_orbsmaxerr1', 'pl_orbsmaxerr2', 'pl_orbsmaxlim', 'pl_orbsmaxstr', 'pl_orbincl', 'pl_orbinclerr1', 'pl_orbinclerr2', 'pl_orbincllim', 'pl_orbinclstr', 'pl_orbtper', 'pl_orbtpererr1', 'pl_orbtpererr2', 'pl_orbtperlim', 'pl_orbtperstr', 'pl_orbeccen', 'pl_orbeccenerr1', 'pl_orbeccenerr2', 'pl_orbeccenlim', 'pl_orbeccenstr', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2', 'pl_eqtlim', 'pl_eqtstr', 'pl_occdep', 'pl_occdeperr1', 'pl_occ

In [ ]:
def query_exoplanet_archive(target_name, planet_id):
    """
    Query NASA Exoplanet Archive for planet parameters.
    Returns period, period_err, T0, and T0_err.
    Handles both letter-based IDs (e.g., 'b', 'c') and TOI numeric IDs (e.g., '0.01', '0.02').
    """
    try:
        # For TOI targets with numeric planet IDs (e.g., TOI-4527 with planet_id 0.01)
        # The full name should be "TOI-4527.01" not "TOI-4527 0.01"
        if 'TOI' in target_name and planet_id.replace('.', '').replace('0', '').isdigit():
            # This is a numeric TOI designation like "0.01"
            # Remove leading "0." to get "01" from "0.01"
            if planet_id.startswith('0.'):
                planet_suffix = planet_id[2:]  # Remove "0." prefix
            else:
                planet_suffix = planet_id
            full_name = f"{target_name}.{planet_suffix}"
        else:
            # Regular letter-based designation (e.g., "TOI-700 b")
            full_name = f"{target_name} {planet_id}".strip()
        
        print(f"  Querying: {full_name}")
        
        # Helper function to extract float value from potentially unit-laden values
        def extract_value(val):
            if val is None:
                return np.nan
            # Handle masked values
            if np.ma.is_masked(val):
                return np.nan
            # Try to get the value if it's a Quantity with units
            if hasattr(val, 'value'):
                try:
                    return float(val.value)
                except:
                    return np.nan
            # Try direct conversion
            try:
                return float(val)
            except:
                return np.nan
        
        # Try confirmed planets table first
        try:
            result = NasaExoplanetArchive.query_object(full_name, table='ps')
            if result is not None and len(result) > 0:
                # Get the first row
                row = result[0]
                data = {
                    'Period (days)': extract_value(row['pl_orbper']) if 'pl_orbper' in result.colnames else np.nan,
                    'P_sig': extract_value(row['pl_orbpererr1']) if 'pl_orbpererr1' in result.colnames else np.nan,
                    'T_mid (BJD)': extract_value(row['pl_tranmid']) if 'pl_tranmid' in result.colnames else np.nan,
                    'T0_sig': extract_value(row['pl_tranmiderr1']) if 'pl_tranmiderr1' in result.colnames else np.nan
                }
                print(f"  Found in confirmed planets catalog")
                return data
        except Exception as e:
            print(f"  Not found in confirmed planets catalog: {e}")
        
        # Try TOI table for candidates not yet confirmed
        if 'TOI' in target_name:
            try:
                # Extract TOI number (e.g., "4527" from "TOI-4527")
                toi_num = target_name.replace('TOI-', '').replace('TOI ', '').strip()
                
                # For numeric planet IDs, construct the full TOI designation
                if planet_id.replace('.', '').replace('0', '').isdigit():
                    # Query by full TOI designation (e.g., "4527.01")
                    if planet_id.startswith('0.'):
                        planet_suffix = planet_id[2:]  # Remove "0." prefix
                    else:
                        planet_suffix = planet_id
                    toi_full = f"{toi_num}.{planet_suffix}"
                    result = NasaExoplanetArchive.query_criteria(table='toi', where=f"toi={toi_full}")
                else:
                    # Query the TOI catalog for this TOI number and match by letter
                    result = NasaExoplanetArchive.query_criteria(table='toi', where=f"toi={toi_num}")
                
                if result is not None and len(result) > 0:
                    # For numeric IDs, take the first result (should be unique)
                    # For letter IDs, find the row matching our planet letter
                    if planet_id.replace('.', '').replace('0', '').isdigit():
                        row = result[0]
                        data = {
                            'Period (days)': extract_value(row['pl_orbper']) if 'pl_orbper' in result.colnames else np.nan,
                            'P_sig': extract_value(row['pl_orbpererr1']) if 'pl_orbpererr1' in result.colnames else np.nan,
                            'T_mid (BJD)': extract_value(row['pl_tranmid']) if 'pl_tranmid' in result.colnames else np.nan,
                            'T0_sig': extract_value(row['pl_tranmiderr1']) if 'pl_tranmiderr1' in result.colnames else np.nan
                        }
                        print(f"  Found in TOI catalog")
                        return data
                    else:
                        # Letter-based: find matching planet letter
                        for row in result:
                            row_planet = str(row['pl_letter']).strip() if 'pl_letter' in result.colnames else ''
                            if row_planet == planet_id.strip():
                                data = {
                                    'Period (days)': extract_value(row['pl_orbper']) if 'pl_orbper' in result.colnames else np.nan,
                                    'P_sig': extract_value(row['pl_orbpererr1']) if 'pl_orbpererr1' in result.colnames else np.nan,
                                    'T_mid (BJD)': extract_value(row['pl_tranmid']) if 'pl_tranmid' in result.colnames else np.nan,
                                    'T0_sig': extract_value(row['pl_tranmiderr1']) if 'pl_tranmiderr1' in result.colnames else np.nan
                                }
                                print(f"  Found in TOI catalog")
                                return data
                        print(f"  Planet {planet_id} not found in TOI {toi_num} results")
            except Exception as e:
                print(f"  Error in TOI catalog: {e}")
        
        print(f"  No data found for {full_name}")
        return None
        
    except Exception as e:
        print(f"  Error querying {full_name}: {e}")
        return None

In [ ]:
# Fill in missing planet data
print("Querying NASA Exoplanet Archive for planet data...\n")

updated_count = 0
skipped_count = 0

for idx, row in df.iterrows():
    # Check if this row has a planet ID
    if pd.notna(row['Planet ID']) and row['Planet ID'].strip() != '':
        # Check if period or T0 is missing
        if pd.isna(row['Period (days)']) or pd.isna(row['T_mid (BJD)']):
            target_name = row['Target Name']
            planet_id = row['Planet ID']
            
            # Skip if target name is empty
            if pd.isna(target_name) or target_name.strip() == '':
                continue
            
            print(f"Processing: {target_name} {planet_id}")
            
            # Query the exoplanet archive
            data = query_exoplanet_archive(target_name, planet_id)
            
            # Update the dataframe if we found data
            if data is not None:
                for key, value in data.items():
                    if pd.isna(row[key]) and not pd.isna(value):
                        df.at[idx, key] = value
                updated_count += 1
        else:
            skipped_count += 1

print(f"\nDone! Updated {updated_count} planets with NASA Exoplanet Archive data.")
print(f"Skipped {skipped_count} planets that already had period and T0 data.")

Querying NASA Exoplanet Archive for planet data...

Processing: TOI-2095 0.03
  Querying: TOI-2095.03


  No data found for TOI-2095.03
Processing: 55 Cnc e
  Querying: 55 Cnc e
  Found in confirmed planets catalog

Done! Updated 1 planets with NASA Exoplanet Archive data.
Skipped 56 planets that already had period and T0 data.
  Found in confirmed planets catalog

Done! Updated 1 planets with NASA Exoplanet Archive data.
Skipped 56 planets that already had period and T0 data.


In [ ]:
# Review the updated planet data
planet_data_updated = df[df['Planet ID'].notna() & (df['Planet ID'] != '')]
print("Planet data with Period and T0:")
print(planet_data_updated[['Target Name', 'Planet ID', 'Period (days)', 'P_sig', 'T_mid (BJD)', 'T0_sig']].head(20))

Planet data with Period and T0:
       Target Name Planet ID  Period (days)         P_sig   T_mid (BJD)  \
0        HD 209458         b       3.524749  3.800000e-07  0.3429948798   
4         TOI-2095      0.03            NaN           NaN           NaN   
5         TOI-2095         b      17.664872  5.100000e-05   2459646.704   
6         TOI-2095         c      28.172200  1.400000e-04   2459662.146   
7          TOI-700         b       9.977020  2.400000e-04   2458490.987   
8          TOI-700         c      16.051110  6.200000e-05   2458548.753   
9          TOI-700         e      27.809690  1.541360e-04       2458353   
10         TOI-700         d      37.423684  1.494550e-04   2458330.486   
56           K2-33         b       5.424865  3.500000e-05   2456898.693   
57  IRAS04125+2902         b       8.834978  2.800000e-05   2458821.825   
58         KELT-11         b       4.736209  3.770000e-06   2455498.959   
60    BD+05 4868 A         b       1.271869  1.000000e-06   2460556.

## Fill Proper Motions from Gaia DR3

Query Gaia DR3 for proper motions using Gaia source IDs.

In [ ]:
# Query Gaia DR3 for proper motions and improved coordinates using Gaia IDs
from astroquery.gaia import Gaia

# Filter rows with Gaia DR3 IDs that are missing proper motions
gaia_rows = df[
    (df['Other name'].notna()) & 
    (df['Other name'] != '') & 
    (df['Other name'].str.isnumeric()) &  # Gaia IDs are numeric
    (df['dRA (mas/yr)'].isna() | df['dDEC (mas/yr)'].isna())  # Missing proper motions
]

if len(gaia_rows) == 0:
    print("No rows need Gaia DR3 updates (all have proper motions already)")
else:
    print(f"Found {len(gaia_rows)} rows with Gaia DR3 IDs needing proper motion data")
    print("Querying Gaia DR3 for proper motions and photometry...\n")
    
    # Query Gaia in batches to avoid timeout
    batch_size = 100
    updated_count = 0
    
    for batch_start in range(0, len(gaia_rows), batch_size):
        batch_end = min(batch_start + batch_size, len(gaia_rows))
        batch = gaia_rows.iloc[batch_start:batch_end]
        
        # Create list of Gaia IDs for this batch
        gaia_ids = ','.join(batch['Other name'].astype(str).tolist())
        
        try:
            # Query Gaia DR3
            query = f"""
            SELECT source_id, ra, dec, pmra, pmdec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag
            FROM gaiadr3.gaia_source
            WHERE source_id IN ({gaia_ids})
            """
            
            job = Gaia.launch_job(query)
            results = job.get_results()
            
            if results is not None and len(results) > 0:
                # Update the dataframe with Gaia data
                for row in results:
                    gaia_id = str(row['source_id'])
                    
                    # Find matching row in df
                    mask = df['Other name'] == gaia_id
                    idx = df[mask].index
                    
                    if len(idx) > 0:
                        idx = idx[0]
                        
                        # Only update if currently missing
                        if pd.isna(df.at[idx, 'RA (deg)']):
                            df.at[idx, 'RA (deg)'] = float(row['ra'])
                        if pd.isna(df.at[idx, 'DEC (deg)']):
                            df.at[idx, 'DEC (deg)'] = float(row['dec'])
                        
                        if not np.ma.is_masked(row['pmra']) and row['pmra'] is not None:
                            if pd.isna(df.at[idx, 'dRA (mas/yr)']):
                                df.at[idx, 'dRA (mas/yr)'] = float(row['pmra'])
                        if not np.ma.is_masked(row['pmdec']) and row['pmdec'] is not None:
                            if pd.isna(df.at[idx, 'dDEC (mas/yr)']):
                                df.at[idx, 'dDEC (mas/yr)'] = float(row['pmdec'])
                        
                        # Use G magnitude if J mag is missing
                        if not np.ma.is_masked(row['phot_g_mean_mag']) and row['phot_g_mean_mag'] is not None:
                            if pd.isna(df.at[idx, 'J_mag']):
                                df.at[idx, 'J_mag'] = float(row['phot_g_mean_mag'])
                        
                        updated_count += 1
            
            print(f"Processed batch {batch_start//batch_size + 1}/{(len(gaia_rows)-1)//batch_size + 1} ({batch_end}/{len(gaia_rows)} stars)")
            
        except Exception as e:
            print(f"Error querying batch starting at {batch_start}: {e}")
            continue
    
    print(f"\nSuccessfully updated {updated_count} rows with Gaia DR3 data")

No rows need Gaia DR3 updates (all have proper motions already)


## Fill Gaia DR3 Identifiers

Query SIMBAD to fill missing Gaia DR3 identifiers in the DR3 column.

In [ ]:
# Check if DR3 column exists, if not create it
if 'DR3' not in df.columns:
    df['DR3'] = np.nan
    print("Created new 'DR3' column")
else:
    print(f"DR3 column already exists with {df['DR3'].notna().sum()} filled entries")

# Configure SIMBAD to return Gaia DR3 IDs
simbad_dr3 = Simbad()
simbad_dr3.add_votable_fields('ids')

# Find rows with missing DR3 identifiers
missing_dr3 = df[df['DR3'].isna() | (df['DR3'] == '')]

if len(missing_dr3) == 0:
    print("No rows need DR3 updates (all have Gaia DR3 IDs already)")
else:
    print(f"Found {len(missing_dr3)} rows with missing Gaia DR3 identifiers")
    print("Querying SIMBAD for Gaia DR3 IDs...\n")
    
    updated_count = 0
    failed_count = 0
    
    for idx, row in missing_dr3.iterrows():
        target_name = row['Target Name']
        
        # Skip empty target names
        if pd.isna(target_name) or target_name.strip() == '':
            continue
        
        # Skip AP stars - they already have Gaia IDs in 'Other name'
        if row['Program ID'] == 'ApStarSurvey':
            # Copy from Other name to DR3 if available
            if pd.notna(row['Other name']) and row['Other name'].strip() != '':
                df.at[idx, 'DR3'] = row['Other name']
                updated_count += 1
            continue
        
        try:
            # Query SIMBAD for IDs
            result = simbad_dr3.query_object(target_name)
            
            if result is None or len(result) == 0:
                # Try with Other name if available
                if pd.notna(row['Other name']) and row['Other name'].strip() != '':
                    other_name = row['Other name']
                    result = simbad_dr3.query_object(other_name)
            
            if result is not None and len(result) > 0:
                # Extract Gaia DR3 ID from the IDS field - check for different column names
                ids_field = None
                
                # Check available columns and get IDS field
                for col in result.colnames:
                    if col.upper() == 'IDS':
                        ids_field = result[col][0]
                        break
                
                if ids_field is None:
                    # Debug: print available columns if IDS field not found
                    if updated_count == 0 and failed_count == 0:
                        print(f"  DEBUG - Available columns: {result.colnames}")
                    failed_count += 1
                elif ids_field and not np.ma.is_masked(ids_field):
                    ids_list = ids_field.split('|')
                    
                    # Look for Gaia DR3 identifier
                    found_gaia = False
                    for id_str in ids_list:
                        if 'Gaia DR3' in id_str:
                            # Extract just the numeric ID
                            gaia_id = id_str.replace('Gaia DR3', '').strip()
                            df.at[idx, 'DR3'] = gaia_id
                            updated_count += 1
                            print(f"  {target_name}: {gaia_id}")
                            found_gaia = True
                            break
                    
                    if not found_gaia:
                        failed_count += 1
                else:
                    failed_count += 1
            else:
                failed_count += 1
                
        except Exception as e:
            print(f"  Error querying {target_name}: {e}")
            failed_count += 1
            continue
    
    print(f"\nSuccessfully filled {updated_count} DR3 identifiers")
    print(f"Failed to find {failed_count} DR3 identifiers")

DR3 column already exists with 2710 filled entries
Found 348 rows with missing Gaia DR3 identifiers
Querying SIMBAD for Gaia DR3 IDs...

  HD 209458: 1779546757669063552
Found 348 rows with missing Gaia DR3 identifiers
Querying SIMBAD for Gaia DR3 IDs...

  HD 209458: 1779546757669063552


/tmp/ipykernel_2189625/62744877.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1779546757669063552' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'DR3'] = gaia_id


  KIC 8462852: 2081900940499099136
  HD 189733: 1827242816201846144
  GJ 411: 762815470562110464
  TOI-2095: 2268372099615724288
  TOI-2095: 2268372099615724288
  TOI-2095: 2268372099615724288
  GJ 411: 762815470562110464
  TOI-2095: 2268372099615724288
  TOI-2095: 2268372099615724288
  TOI-2095: 2268372099615724288
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752
  TOI-700: 5284517766615850752


  AU Mic: 6794047652729201024
  Proxima: 5853498713190525696
  EV Lac: 1934263333784036736
  Wolf 359: 3864972938605115520
  EV Lac: 1934263333784036736
  Wolf 359: 3864972938605115520
  YZ CMi: 3136952686035250688
  Barnard's Star: 4472832130942575872
  YZ CMi: 3136952686035250688
  Barnard's Star: 4472832130942575872
  HD 101065: 5372587514128271232
  HD 37519: 3447703809100726400
  HD 101065: 5372587514128271232
  HD 37519: 3447703809100726400
  HD 95735: 762815470562110464
  BD+44 2051: 778947814402602752
  HD 95735: 762815470562110464
  BD+44 2051: 778947814402602752
  HD 215441: 2003845308681117824
  61 Cygni A: 1872046609345556480
  HD 215441: 2003845308681117824
  61 Cygni A: 1872046609345556480
  HD 125823: 6116497702589633152
  HD 30353: 204531088580182016
  HD 125823: 6116497702589633152
  HD 30353: 204531088580182016
  HD 127617: 1239086696118147712
  HD 127617: 1239086696118147712
  HD 50169: 3106283797745657216
  HD 6090: 4685972945383736064
  HD 50169: 310628379774565721

## Save Final Results

Save the completed dataframe with all updates to a new CSV file.

In [ ]:
# Save the final updated dataframe
output_file = 'Pandora Snapshot Targets - Targets_updated.csv'
df.to_csv(output_file, index=False)
print(f"Saved final updated data to '{output_file}'")

# Show summary statistics
print(f"\nFinal dataframe summary:")
print(f"  Total targets: {len(df)}")
print(f"\nProgram IDs:")
for program, count in df['Program ID'].value_counts().items():
    print(f"  {program}: {count} targets")

print(f"\nData completeness:")
print(f"  Rows with RA/DEC: {df['RA (deg)'].notna().sum()}/{len(df)}")
print(f"  Rows with proper motions: {df['dRA (mas/yr)'].notna().sum()}/{len(df)}")
print(f"  Rows with J magnitude: {df['J_mag'].notna().sum()}/{len(df)}")
print(f"  Planet rows with period: {df[df['Planet ID'].notna()]['Period (days)'].notna().sum()}/{df['Planet ID'].notna().sum()}")

Saved final updated data to 'Pandora Snapshot Targets - Targets_updated.csv'

Final dataframe summary:
  Total targets: 3058

Program IDs:
  ApStarSurvey: 2710 targets
  HWO-tier-1: 164 targets
  GEMS-Targets: 55 targets
  Exotic-Stars: 31 targets
  TBOSS: 18 targets
  Volcanic-Atmospheres: 15 targets
  HJs-with-Friends: 9 targets
  Flare-Stars: 7 targets
  BD Lightcurve: 5 targets
  Kepler-targets: 4 targets
  Circumbinary-Planets: 4 targets
  M+T pairs: 4 targets
  TOI-700-TTVs: 4 targets
  GEMs-Hosts: 4 targets
  TOI-2095 TTVs: 3 targets
  Famous Planets: 3 targets
  Phase Curves: 3 targets
  Young-Planets: 2 targets
  Solar-system: 2 targets
  Inflated-Planets: 2 targets
  Atmospheric-Escape: 2 targets
  TOI-700-general: 1 targets
  TBOSS young-ish M+L dwarfs in Taurus region with ALMA obs: 1 targets
  Sweet-Planets: 1 targets
  Disintegrating-Planets: 1 targets
  benchmark M+T pairs: 1 targets

Data completeness:
  Rows with RA/DEC: 3058/3058
  Rows with proper motions: 3037/3058
